In [1]:
from gym_match3.envs import Match3Env
import gym
import matplotlib.pyplot as plt

from matplotlib import style
import numpy as np

env = Match3Env()
#obs, reward, done, info = env.step(0) 
import torch
import cv2

from PIL import Image

from gym_match3.envs.levels import LEVELS #  default levels
from gym_match3.envs.levels import Match3Levels, Level


from gym_match3.envs.game import (Board,
                                  RandomBoard,
                                  CustomBoard,
                                  Point,
                                  Cell,
                                  AbstractSearcher,
                                  MatchesSearcher)


from gym_match3.envs.game import OutOfBoardError, ImmovableShapeError

def one_hot_encoding(obs):
    
    obs = obs.reshape(obs.shape[1],obs.shape[2]).astype(int)
    
    grid_onehot = np.zeros(shape=(env.n_shapes, obs.shape[0], obs.shape[1]))
    table = {i:i for i in range(0,10)}
    
    for i in range(obs.shape[0]):
        for j in range(obs.shape[1]):
            grid_element = obs[i][j]
            grid_onehot[table[grid_element]][i][j]=1
            
    return grid_onehot



    
    
def Getlevels(WidthAndHeight,shapes):
    LEVELS = [Level(WidthAndHeight,WidthAndHeight,shapes, np.zeros((WidthAndHeight,WidthAndHeight)).tolist())]
    return LEVELS




In [4]:
from gym_match3.envs import Match3Env
import gym
from PIL import Image
import cv2
import numpy as np
from random import choice


env = Match3Env(levels=Match3Levels(Getlevels(5,5)))



available_actions = {v : k for k, v in dict(enumerate(env.get_available_actions())).items()}



for i_episode in range(1): #玩 1次遊戲
     
    observation = env.reset() 
    
    while True:
        env.render()
       
        validate_move = env.possible_move   # 一般用 env 的屬性紀錄合法走步
        
        
        
        #一開始遊戲初始化時,env的屬性會是 None,以及重新一場遊戲時要再 get一次合法走步
        if validate_move == None or len(validate_move)== 0:  
      
            validate_move = env.get_validate_actions()
        
        #print(validate_move)
        
        validate_list = []       
        for i in validate_move:
            if i in available_actions:
                validate_list.append(available_actions.get(i))   
         

        action = choice(validate_list)
        
        
        observation, reward, done, info = env.step(action)  # step函數會檢查下一個 observation 有沒有合法走步,並回傳 observation,如果沒有合法走步 done == True
            
        
        print('the swap of coordinate is: ',list(env.get_available_actions()[action]))

        if done:           
            break


the swap of coordinate is:  [(1, 3), (1, 4)]
the swap of coordinate is:  [(0, 2), (1, 2)]
the swap of coordinate is:  [(2, 1), (2, 2)]
the swap of coordinate is:  [(2, 0), (3, 0)]
the swap of coordinate is:  [(1, 2), (2, 2)]
the swap of coordinate is:  [(2, 0), (2, 1)]
the swap of coordinate is:  [(3, 0), (3, 1)]
the swap of coordinate is:  [(0, 4), (1, 4)]
the swap of coordinate is:  [(2, 0), (2, 1)]
the swap of coordinate is:  [(2, 2), (3, 2)]
the swap of coordinate is:  [(2, 3), (2, 4)]
the swap of coordinate is:  [(3, 3), (3, 4)]
the swap of coordinate is:  [(0, 3), (1, 3)]
the swap of coordinate is:  [(0, 2), (1, 2)]
the swap of coordinate is:  [(1, 2), (2, 2)]
the swap of coordinate is:  [(4, 1), (4, 2)]
the swap of coordinate is:  [(0, 2), (0, 3)]
the swap of coordinate is:  [(2, 0), (2, 1)]
